Follow the instructions before running the code:
    
https://github.com/unmannedlab/RELLIS-3D/tree/979a533262b77a29901d1fa6f747b39600d4ffd3/benchmarks

In [1]:
import os
import sys
sys.path.append('../../RELLIS-3D/benchmarks/HRNet-Semantic-Segmentation-HRNet-OCR/lib/')
import torch

In [2]:
from config import config
from config import update_config
from core.function import testval, test
from utils.modelsummary import get_model_summary
from utils.utils import create_logger, FullModel
import models

In [3]:
model_state_file = "../config/weights/seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484/best.pth"

In [4]:
cfg_path = "../../RELLIS-3D/benchmarks/HRNet-Semantic-Segmentation-HRNet-OCR/experiments/rellis/seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-3_wd5e-4_bs_12_epoch484.yaml"

config.defrost()
config.merge_from_file(cfg_path)
config.merge_from_list(['TEST.MODEL_FILE',
                       '/home/ruslan/workspaces/traversability_ws/src/traversability_estimation/config/weights/seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484/best.pth'])
config.freeze()

In [5]:
model = eval('models.' + config.MODEL.NAME + '.get_seg_model')(config)

{'conv3x3_ocr.1.num_batches_tracked', 'ocr_distri_head.conv_bn_dropout.1.0.bias', 'aux_head.1.running_var', 'ocr_distri_head.object_context_block.f_object.1.0.weight', 'ocr_distri_head.object_context_block.f_pixel.3.0.running_var', 'ocr_distri_head.object_context_block.f_up.1.0.running_mean', 'ocr_distri_head.object_context_block.f_up.1.0.num_batches_tracked', 'ocr_distri_head.conv_bn_dropout.0.weight', 'ocr_distri_head.object_context_block.f_pixel.1.0.running_mean', 'aux_head.3.weight', 'ocr_distri_head.object_context_block.f_object.1.0.bias', 'aux_head.0.bias', 'ocr_distri_head.object_context_block.f_pixel.1.0.running_var', 'ocr_distri_head.conv_bn_dropout.1.0.running_var', 'ocr_distri_head.object_context_block.f_object.1.0.num_batches_tracked', 'cls_head.weight', 'ocr_distri_head.object_context_block.f_pixel.1.0.num_batches_tracked', 'conv3x3_ocr.1.running_mean', 'ocr_distri_head.object_context_block.f_pixel.3.0.weight', 'ocr_distri_head.object_context_block.f_up.1.0.weight', 'ocr_d

In [6]:
pretrained_dict = torch.load(model_state_file)
if 'state_dict' in pretrained_dict:
    pretrained_dict = pretrained_dict['state_dict']
model_dict = model.state_dict()
pretrained_dict = {k[6:]: v for k, v in pretrained_dict.items()
                    if k[6:] in model_dict.keys()}
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

<All keys matched successfully>

In [7]:
from torch import nn

gpus = list(config.GPUS)
print('GPUS:',gpus)
model = nn.DataParallel(model, device_ids=gpus).cuda()

GPUS: [0]


In [8]:
import datasets

# prepare data
test_size = (config.TEST.IMAGE_SIZE[1], config.TEST.IMAGE_SIZE[0])
test_dataset = eval('datasets.'+config.DATASET.DATASET)(
                    root=config.DATASET.ROOT,
                    list_path=config.DATASET.TEST_SET,
                    num_samples=None,
                    num_classes=config.DATASET.NUM_CLASSES,
                    multi_scale=False,
                    flip=False,
                    ignore_label=config.TRAIN.IGNORE_LABEL,
                    base_size=config.TEST.BASE_SIZE,
                    crop_size=test_size,
                    downsample_rate=1)

testloader = torch.utils.data.DataLoader(test_dataset,
                                         batch_size=1,
                                         shuffle=False,
                                         num_workers=config.WORKERS,
                                         pin_memory=True)

In [9]:
import yaml

data_cfg = "../../RELLIS-3D/benchmarks/SalsaNext/train/tasks/semantic/config/labels/rellis.yaml"
CFG = yaml.safe_load(open(data_cfg, 'r'))
id_color_map = CFG["color_map"]

In [18]:
from tqdm import tqdm
from core.function import convert_label, convert_color
from PIL import Image
import numpy as np

viz = True
model.eval()

with torch.no_grad():
    
    for it, batch in enumerate(tqdm(testloader)):
        
        if it == 5:
            break
        
        image, size, name = batch
        print(image.device)
        
        size = size[0]
        pred = model(image)
        pred = pred[config.TEST.OUTPUT_INDEX]
        pred_np = pred.cpu().numpy()
        b,_,_,_ = pred.shape
        
        for i in range(b):
            sv_path = os.path.join(config.OUTPUT_DIR, 'hrnet',name[i][:5],'pylon_camera_node_label_id')
            if not os.path.exists(sv_path):
                os.makedirs(sv_path)
            _, file_name = os.path.split(name[i])
            file_name = file_name.replace("jpg","png")
            data_path = os.path.join(sv_path,file_name)
            pred_arg = np.argmax(pred_np[i],axis=0).astype(np.uint8)
            pred_arg = convert_label(pred_arg, True)
            pred_img = np.stack((pred_arg,pred_arg,pred_arg),axis=2)
            pred_img = Image.fromarray(pred_img)
            pred_img.save(data_path)
            
            if viz:
                sv_path = os.path.join(config.OUTPUT_DIR, 'hrnet',name[i][:5],'pylon_camera_node_label_color')
                if not os.path.exists(sv_path):
                    os.makedirs(sv_path)
                _, file_name = os.path.split(name[i])
                file_name = file_name.replace("jpg","png")
                color_path = os.path.join(sv_path,file_name)
                color_label = convert_color(pred_arg, id_color_map)

                color_img = Image.fromarray(color_label,'RGB')
                color_img.save(color_path)

  0%|                                                                                                                                               | 1/1672 [00:00<08:51,  3.14it/s]

cpu
cpu


  0%|▎                                                                                                                                              | 3/1672 [00:00<04:23,  6.33it/s]

cpu
cpu


  0%|▍                                                                                                                                              | 5/1672 [00:00<03:38,  7.62it/s]

cpu


  0%|▍                                                                                                                                              | 5/1672 [00:00<05:00,  5.55it/s]
